### LVV-T128: Verify implementation of Provide Astrometric Model

Verify that an astrometric model is available for Objects and DIAObjects, and that it includes the proper motion and parallax, and the estimated uncertainties on these quantities.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from lsst.daf.butler import Butler

In [3]:
butler = Butler('dp2_prep', collections=['LSSTCam/runs/DRP/20250417_20250921/w_2025_49/DM-53545'])

The outputs of `gbdes` are stored in "healpix3" sky pixels. Query the butler to find datasets produced by `gbdes`.

In [4]:
butler.registry.queryDatasetTypes('*Healpix3*')

[DatasetType('gbdesHealpix3AstrometricFit_config', {}, Config),
 DatasetType('gbdesHealpix3AstrometricFitSkyWcsCatalog', {band, healpix3, instrument, day_obs, physical_filter, visit}, ExposureCatalog),
 DatasetType('gbdesHealpix3AstrometricFit_fitStars', {band, healpix3, instrument, physical_filter}, ArrowNumpyDict),
 DatasetType('gbdesHealpix3AstrometricFit_log', {band, healpix3, instrument, physical_filter}, ButlerLogRecords),
 DatasetType('gbdesHealpix3AstrometricFit_metadata', {band, healpix3, instrument, physical_filter}, TaskMetadata),
 DatasetType('gbdesHealpix3AstrometricFit_starCatalog', {band, healpix3, instrument, physical_filter}, ArrowNumpyDict),
 DatasetType('gbdesHealpix3AstrometricFit_gatherResourceUsage_config', {}, Config),
 DatasetType('gbdesHealpix3AstrometricFit_gatherResourceUsage_log', {}, ButlerLogRecords),
 DatasetType('gbdesHealpix3AstrometricFit_gatherResourceUsage_metadata', {}, TaskMetadata),
 DatasetType('gbdesHealpix3AstrometricFit_resource_usage', {}, Da

Inspect the `gbdesHealpix3AstrometricFit_starCatalog`:

In [5]:
gbdes_refs = butler.query_datasets('gbdesHealpix3AstrometricFit_starCatalog', band='r')
print(gbdes_refs[0])
gbdes_cat = butler.get(gbdes_refs[0])

gbdesHealpix3AstrometricFit_starCatalog@{healpix3: 530, instrument: 'LSSTCam', physical_filter: 'r_57', band: 'r'} [sc=ArrowNumpyDict] (run=LSSTCam/runs/DRP/20250417_20250921/w_2025_49/DM-53545/20251206T220256Z id=019af5b0-e144-7951-b7d3-b1aaf9ac8652)


In [6]:
gbdes_cat

{'starMatchID': array([    0,     1,     2, ..., 63549, 63550, 63551],
       shape=(63318,), dtype=int32),
 'starReserve': array([False, False, False, ...,  True, False,  True], shape=(63318,)),
 'starColor': array([0.61, 0.61, 0.61, ..., 0.61, 0.61, 0.61], shape=(63318,)),
 'starPMCount': array([1, 1, 1, ..., 1, 1, 1], shape=(63318,), dtype=int32),
 'starDetCount': array([3, 3, 3, ..., 1, 0, 1], shape=(63318,), dtype=int32),
 'starClipCount': array([0, 0, 0, ..., 0, 1, 0], shape=(63318,), dtype=int32),
 'starDOF': array([6, 6, 6, ..., 2, 0, 2], shape=(63318,), dtype=int32),
 'starChisq': array([120.79367947,  20.37949231,  14.91082506, ..., 103.77243073,
          0.        , 635.03637611], shape=(63318,)),
 'starX': array([59.52734051, 59.53205387, 59.54014731, ..., 66.72735348,
        66.74603663, 66.75315704], shape=(63318,)),
 'ra': array([59.52734051, 59.53205387, 59.54014731, ..., 66.72735348,
        66.74603663, 66.75315704], shape=(63318,)),
 'starY': array([-48.50417954, -

We see that this catalog contains the fit proper motions and parallaxes for 63318 stars.

Now inspect the final proper motion catalog, known as `isolated_star_stellar_motions`:

In [7]:
pmcat_refs = butler.query_datasets('isolated_star_stellar_motions',
                                   tract=10321, skymap='lsst_cells_v2')
print(pmcat_refs)

[DatasetRef(DatasetType('isolated_star_stellar_motions', {instrument, skymap, tract}, ArrowAstropy), {instrument: 'LSSTCam', skymap: 'lsst_cells_v2', tract: 10321}, run='LSSTCam/runs/DRP/20250417_20250921/w_2025_49/DM-53545/20251210T220225Z', id=019b0a4a-05b1-7dc7-951f-bb3b2a4ae92c)]


In [8]:
pmcat = butler.get(pmcat_refs[0])

In [9]:
pmcat.colnames

['ra',
 'dec',
 'raPM',
 'decPM',
 'parallax',
 'hasReference',
 'covariance',
 'ref_ra',
 'ref_dec',
 'ref_raPM',
 'ref_decPM',
 'ref_covariance',
 'isolated_star_id']

In [10]:
pmcat[:5]

ra,dec,raPM,decPM,parallax,hasReference,covariance,ref_ra,ref_dec,ref_raPM,ref_decPM,ref_covariance,isolated_star_id
float64,float64,float64,float64,float64,bool,"float64[5,5]",float64,float64,float64,float64,"float64[5,5]",int64
-174.36394708791187,4.765773653581895,-19.511120322329052,-3.483486153571007,0.29211075053273805,True,0.3458563523861943 .. 0.4842031466316532,185.63605432297373,4.765774411072197,-19.517689680809028,-3.4749931320599883,60.423439025878906 .. 0.49247419943680354,110321
-174.3385928482932,4.691719016874777,-3.0926701799249368,-7.970446693759035,0.7629205686624485,True,0.4745778495714253 .. 0.7540405811574453,185.66140214733565,4.691715030455214,-3.088645608584405,-7.951608066757335,78.1109390258789 .. 0.7747987793249024,210321
-174.42026417741704,4.700767323604473,1.4278604208496148,-0.8365128484227922,-0.7202958735506392,True,0.5284870916906211 .. 1.0863956859729273,185.5797389984859,4.700767941050039,1.5154352948079033,-0.7780315816333618,194.05010986328125 .. 1.1200632355641595,310321
-174.2595767248721,4.680545960362633,17.593937752190573,6.29391555536048,-0.6485894644354723,False,72.5991377817475 .. 86.18482629013003,nan,nan,nan,nan,nan .. nan,410321
-174.42363335921857,4.727347083701902,8.84200264238125,-5.675328720738012,2.4955296888034657,True,0.2569614197068057 .. 0.3537297448944969,185.57636748211922,4.7273425270769085,8.808864331977725,-5.575198357820922,40.80447006225586 .. 0.36134818787155354,510321


### Results

We see that proper motion and parallax are being measured for "isolated stars" (the subset of stars that are selected for calibration purposes), and stored in a dataset type called `isolated_star_stellar_motions`. Covariances are also reported, implying that we can easily derive uncertainties on the measurements.

Because only a subset of objects are currently being measured, and because they are not matched to the `Object` and `DIAObject` tables, there is still work to be done. Nonetheless, we have demonstrated that the tools to make the measurements exist.